# 🚀 KLT GPU Integration Testing

This notebook tests the **REAL GPU integration** where the modified `convolve.c` file actually calls GPU functions during KLT algorithm execution.

## 🎯 What This Tests:
- **Real GPU integration** in the KLT algorithm
- **Modified convolve.c** that calls GPU functions
- **Performance comparison** (CPU vs GPU speedup)
- **Verification** that GPU functions are actually used

## 📋 Prerequisites:
- CUDA-enabled GPU (Tesla T4, V100, A100, etc.)
- Modified `convolve.c` file uploaded
- GPU functions (`convolve_gpu_functions.cu`) uploaded
- All KLT source files uploaded


## 🔧 Step 1: Setup and File Verification


In [ ]:
# Setup and file verification
import os
import shutil
import subprocess
import time
import numpy as np

print("🚀 KLT GPU Integration Testing Setup")
print("===================================")

# Check CUDA availability
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, check=True)
    print("✅ CUDA GPU detected:")
    print(result.stdout)
except subprocess.CalledProcessError:
    print("❌ No CUDA GPU detected. Please enable GPU in Runtime > Change runtime type > GPU")
    exit()

# Check if required files exist
required_files = [
    'klt/src/convolve.c',  # Modified to call GPU functions
    'klt/src/convolve_gpu_functions.cu',  # GPU implementations
    'klt/src/example3.c',
    'klt/src/klt.c',
    'klt/include/klt_util.h'
]

print("\n📁 Checking required files...")
missing_files = []
for file in required_files:
    if os.path.exists(file):
        print(f"✅ {file}")
    else:
        print(f"❌ {file} - MISSING!")
        missing_files.append(file)

if missing_files:
    print(f"\n⚠️  Missing {len(missing_files)} files. Please upload all required files.")
    print("Required files:")
    for file in missing_files:
        print(f"  - {file}")
else:
    print("\n✅ All required files found!")

print("\n🎯 Ready to test real GPU integration!")


## 🔧 Step 2: Compile KLT with GPU Integration


In [ ]:
# Compile KLT with real GPU integration
print("🔧 Compiling KLT with Real GPU Integration")
print("=========================================")

# Compile KLT with the modified convolve.c and GPU functions
klt_gpu_cmd = [
    'nvcc',
    '-O3',
    '-std=c++11',
    '-arch=sm_75',  # Tesla T4 architecture
    '-I./klt/include',
    '-o', 'example3_gpu_integrated',
    'klt/src/example3.c',
    'klt/src/klt.c',
    'klt/src/convolve.c',  # Modified convolve.c with GPU calls
    'klt/src/convolve_gpu_functions.cu',  # GPU functions
    'klt/src/error.c',
    'klt/src/pnmio.c',
    'klt/src/pyramid.c',
    'klt/src/selectGoodFeatures.c',
    'klt/src/storeFeatures.c',
    'klt/src/trackFeatures.c',
    'klt/src/klt_util.c',
    'klt/src/writeFeatures.c',
    '-lm'
]

print(f"Compile command: {' '.join(klt_gpu_cmd)}")

try:
    result = subprocess.run(klt_gpu_cmd, capture_output=True, text=True, check=True)
    print("✅ KLT with real GPU integration compiled successfully!")
    if result.stdout:
        print("Output:", result.stdout)
except subprocess.CalledProcessError as e:
    print(f"❌ Compilation failed: {e}")
    print(f"Error: {e.stderr}")
    
    # Try alternative compilation
    print("\n🔄 Trying alternative compilation...")
    alt_cmd = [
        'nvcc',
        '-O3',
        '-std=c++11',
        '-arch=sm_75',
        '-I./klt/include',
        '-o', 'example3_gpu_alt',
        'klt/src/example3.c',
        'klt/src/klt.c',
        'klt/src/convolve.c',
        'klt/src/convolve_gpu_functions.cu',
        'klt/src/error.c',
        'klt/src/pnmio.c',
        'klt/src/pyramid.c',
        'klt/src/selectGoodFeatures.c',
        'klt/src/storeFeatures.c',
        'klt/src/trackFeatures.c',
        'klt/src/klt_util.c',
        'klt/src/writeFeatures.c',
        '-lm'
    ]
    
    try:
        result = subprocess.run(alt_cmd, capture_output=True, text=True, check=True)
        print("✅ Alternative compilation successful!")
    except subprocess.CalledProcessError as e2:
        print(f"❌ Alternative compilation also failed: {e2}")
        print(f"Error: {e2.stderr}")

print("\n🎯 Compilation completed!")


## 🧪 Step 3: Run KLT with GPU Integration
